In [1]:
from torch.utils.data import DataLoader
from lightly.data import LightlyDataset
import SimPLR
import torch

/home/tvanzyl/miniforge3/envs/saccadic-jepa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:
method = "SimPLR"
train_dir = "/media/tvanzyl/data/tiny-imagenet-200/train"
batch_size_per_device = 256
num_workers = 1
ckpt_path = "./benchmark_logs/SimPLR/2025-03-25_17-57-47/pretrain/version_0/checkpoints/epoch=199-step=78000.ckpt"
num_classes=200
resnetsize=18

In [47]:
METHODS = {
    "SimPLR": {"model": SimPLR.SimPLR, "transform": SimPLR.transform},
}
model = METHODS[method]["model"](
            batch_size_per_device=batch_size_per_device, num_classes=num_classes, resnetsize=resnetsize
        )
train_transform = METHODS[method]["transform"]

train_dataset = LightlyDataset(input_dir=str(train_dir), transform=train_transform)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size_per_device,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True,
    persistent_workers=False,
)


In [48]:
model.load_state_dict(torch.load(ckpt_path)["state_dict"])
model.eval();

/tmp/ipykernel_1754610/3608797432.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_path)["state_dict"])


In [5]:
x, targets, _ = train_dataloader.__iter__().__next__()

In [6]:
x[0].shape

torch.Size([256, 3, 224, 224])

In [ ]:
f0 = model.backbone(x[0]).flatten(start_dim=1)
f1 = model.backbone(x[1]).flatten(start_dim=1)

In [57]:
model.criterion(g0, g1)

tensor(-0.9030, grad_fn=<NegBackward0>)

In [8]:
g0 = model.projection_head(f0)
g1 = model.projection_head(f1)

In [9]:
z0 = model.merge_head(g0)
z1 = model.merge_head(g1)

In [10]:
p0 = model.prediction_head(g0)
p1 = model.prediction_head(g1)

In [11]:
model.criterion(p0, z1)

tensor(-1.0000, grad_fn=<NegBackward0>)

In [ ]:
model.criterion(p1, z0)

tensor(-1.0000, grad_fn=<NegBackward0>)

In [83]:
model.criterion(SimPLR.L2NormalizationLayer().forward(model.prediction_head(g0)),model.merge_head(g0*0.0))

tensor(-1.0000, grad_fn=<NegBackward0>)

In [89]:
SimPLR.L2NormalizationLayer().forward(0.0*model.prediction_head(g0))/2.4

tensor([[0., 0., -0.,  ..., -0., 0., 0.],
        [0., 0., -0.,  ..., -0., 0., 0.],
        [0., 0., -0.,  ..., -0., 0., 0.],
        ...,
        [0., 0., -0.,  ..., -0., 0., 0.],
        [0., 0., -0.,  ..., -0., 0., 0.],
        [0., 0., -0.,  ..., -0., 0., 0.]], grad_fn=<DivBackward0>)

In [84]:
model.merge_head(g1*100.0)

tensor([[ 0.0251,  0.0407, -0.0356,  ..., -0.0188, -0.0062,  0.0054],
        [ 0.0379,  0.0087, -0.0107,  ..., -0.0078, -0.0073,  0.0118],
        [ 0.0385,  0.0042, -0.0040,  ..., -0.0066, -0.0119,  0.0148],
        ...,
        [ 0.0390,  0.0048, -0.0008,  ..., -0.0037, -0.0118,  0.0150],
        [ 0.0308,  0.0339, -0.0362,  ..., -0.0197, -0.0032,  0.0066],
        [ 0.0389,  0.0124, -0.0153,  ..., -0.0095, -0.0049,  0.0116]],
       grad_fn=<AddmmBackward0>)

In [85]:
model.merge_head(g0*100.0)

tensor([[ 0.0237,  0.0430, -0.0356,  ..., -0.0189, -0.0074,  0.0044],
        [ 0.0374,  0.0072, -0.0094,  ..., -0.0077, -0.0083,  0.0124],
        [ 0.0382,  0.0041, -0.0059,  ..., -0.0082, -0.0115,  0.0146],
        ...,
        [ 0.0384,  0.0075, -0.0047,  ..., -0.0045, -0.0089,  0.0125],
        [ 0.0341,  0.0270, -0.0301,  ..., -0.0161, -0.0025,  0.0082],
        [ 0.0385,  0.0066, -0.0054,  ..., -0.0050, -0.0098,  0.0130]],
       grad_fn=<AddmmBackward0>)

In [13]:
f_, p, z = model.forward_student( x )

In [14]:
l = 0
for i in range(8):
    print(model.criterion(p[i], z[i]) / model.ens_size)
    l += model.criterion(p[i], z[i]) / model.ens_size


tensor(-0.1250, grad_fn=<DivBackward0>)
tensor(-0.1250, grad_fn=<DivBackward0>)
tensor(-0.1250, grad_fn=<DivBackward0>)
tensor(-0.1250, grad_fn=<DivBackward0>)
tensor(-0.1250, grad_fn=<DivBackward0>)
tensor(-0.1250, grad_fn=<DivBackward0>)
tensor(-0.1250, grad_fn=<DivBackward0>)
tensor(-0.1250, grad_fn=<DivBackward0>)


In [15]:
l

tensor(-1.0000, grad_fn=<AddBackward0>)